## Preparing datasets for visualization

We want to build a visualization in Tableau that would show a timeline with the number of papers in each topic per year, some big numbers with the total counts and a word cloud showing the most important words for each topic.

In order to do so, we are going to create 2 file sources:

 - In the first one we will take our csv created with all our paper abstracts and the assigned topic identified through our LDA model. As we dont need the full abstract take, this file will be just a subset of the original file with all the columns excepting the one with the abstract text. This will serve us to create the timeline and the big numbers.
 
 - In the second file we will have the topic assigned and the top 10 words in each one by their weights in the distribution. This is going to be the source for the word cloud viz.

Lets start with the first file.

In [23]:
import pandas as pd
from gensim.test.utils import datapath
from gensim.models import LdaModel
import re

In [13]:
df_abstracts_topics = pd.read_csv("./papers_abstracts_year_topics.csv")

In [14]:
df_abstracts_topics.head()

,Publication Id,Publication Abstract,year,Assigned Topic
0,1117506301,Four types of calcined MCM-41 silica nanoparti...,2019.0,3.0
1,1117603937,First magnetic characterization of a recently ...,2019.0,3.0
2,1117505850,Metal oxide nanoparticles of different nature ...,2019.0,3.0
3,1120933764,The objective of this study is to identify bot...,2019.0,8.0
4,1117193292,A long-term multi-parameter skill assessment o...,2019.0,6.0


In [15]:
df_abstracts_topics.columns

Index(['Publication Id', 'Publication Abstract', 'year', 'Assigned Topic'], dtype='object')

Lets create the dataframe without the full abstract text and save it to a csv file.

In [16]:
df_abstracts_topics_viz = df_abstracts_topics[['Publication Id', 'year', 'Assigned Topic']]

In [19]:
df_abstracts_topics_viz.head()

,Publication Id,year,Assigned Topic
0,1117506301,2019.0,3.0
1,1117603937,2019.0,3.0
2,1117505850,2019.0,3.0
3,1120933764,2019.0,8.0
4,1117193292,2019.0,6.0


In [18]:
df_abstracts_topics_viz.to_csv("./papers_year_topics_viz.csv", sep=',', index=False )

Lets now create our second file that will feed the word cloud.

First we need to load our LDA model with 10 topics previously saved when generating it

In [20]:
model_file_10 = datapath("model_lda_10")
# Load model
lda = LdaModel.load(model_file_10)

We are going to need to create a dataframe in a form of 3 columns containing:

 - First column : Assigned Topic. With this we will be able to later in Tableau, make a relationship between the first file and this one, so we can create a dynamic dashboard.
 
 - Second column:  Top 10 words for each topic.
 
 - Third column:  Weight of distribution of each word in each topic. So then we can play with this weights to assigned size/color to the word in the word cloud visualization.

In order to create a dataframe with these columns we can use the previous function for displaying the topics of the model, we are going to tweak it a bit to get an output that makes the creation of the dataframe easier, so we can take the entire output lists of words and weigths for each topic and copy/paste as the entries of our dataframe.

In [34]:
def display_topics(model, model_type="lda"):
    for topic_idx, topic in enumerate(model.print_topics( num_words=20)):
        print (topic_idx)
        print(re.findall( r'\d+\.\d+', topic[1]))
        print (re.findall( r'\"(.[^"]+).?', topic[1]))

In [35]:
topics = display_topics(lda)

0
['0.030', '0.010', '0.009', '0.008', '0.008', '0.007', '0.007', '0.007', '0.007', '0.007', '0.007', '0.007', '0.006', '0.006', '0.005', '0.005', '0.005', '0.005', '0.005', '0.005']
['species', 'water', 'areas', 'climate', 'results', 'area', 'data', 'soil', 'populations', 'diversity', 'distribution', 'conditions', 'patterns', 'sites', 'variability', 'region', 'change', 'changes', 'population', 'period']
1
['0.018', '0.012', '0.009', '0.009', '0.007', '0.007', '0.007', '0.006', '0.006', '0.006', '0.006', '0.006', '0.006', '0.006', '0.005', '0.005', '0.005', '0.005', '0.005', '0.005']
['brain', 'results', 'activity', 'effects', 'memory', 'test', 'differences', 'effect', 'muscle', 'changes', 'motor', 'time', 'pain', 'subjects', 'neurons', 'stress', 'exercise', 'response', 'age', 'function']
2
['0.016', '0.011', '0.011', '0.010', '0.010', '0.009', '0.009', '0.008', '0.008', '0.008', '0.008', '0.007', '0.007', '0.007', '0.007', '0.006', '0.006', '0.006', '0.006', '0.006']
['levels', 'weigh

Now we can just take the outputs of our function and type them as the entries of our dataframe.

In [36]:
### create dataframe for word cloud

df_word_cloud_viz = pd.DataFrame(
      {'Assigned Topic' : [0]*20 + [1]*20 + [2]*20 + [3]*20 + [4]*20 + [5]*20 + [6]*20 + [7]*20 + [8]*20 + [9]*20
      ,'Word' : ['species', 'water', 'areas', 'climate', 'results', 'area', 'data', 'soil', 'populations', 'diversity', 'distribution', 'conditions', 'patterns', 'sites', 'variability', 'region', 'change', 'changes', 'population', 'period' 
                ,'brain', 'results', 'activity', 'effects', 'memory', 'test', 'differences', 'effect', 'muscle', 'changes', 'motor', 'time', 'pain', 'subjects', 'neurons', 'stress', 'exercise', 'response', 'age', 'function'
                ,'levels', 'weight', 'body', 'results', 'risk', 'age', 'samples', 'diet', 'obesity', 'children', 'blood', 'resistance', 'groups', 'methods', 'women', 'food', 'population', 'intake', 'strains', 'studies'
                ,'properties', 'surface', 'temperature', 'results', 'energy', 'structure', 'materials', 'phase', 'reaction', 'model', 'electron', 'state', 'formation', 'nanoparticles', 'effect', 'field', 'density', 'measurements', 'process', 'material'
                ,'patients', 'treatment', 'risk', 'years', 'disease', 'methods', 'results', 'cancer', 'cases', 'months', 'conclusions', 'group', 'age', 'therapy', 'diagnosis', 'mortality', 'patient', 'survival', 'surgery', 'rate'
                ,'research', 'students', 'results', 'information', 'knowledge', 'article', 'process', 'development', 'training', 'studies', 'language', 'review', 'group', 'learning', 'data', 'education', 'work', 'literature', 'approach', 'order'
                ,'model', 'data', 'system', 'results', 'time', 'models', 'systems', 'performance', 'energy', 'order', 'approach', 'problem', 'power', 'methods', 'network', 'number', 'algorithm', 'information', 'parameters', 'case'
                ,'cells', 'cell', 'expression', 'protein', 'gene', 'genes', 'activity', 'role', 'proteins', 'levels', 'results', 'mice', 'response', 'activation', 'receptor', 'tissue', 'cancer', 'effects', 'mechanisms', 'development'
                ,'health', 'results', 'countries', 'factors', 'women', 'quality', 'care', 'life', 'sample', 'variables', 'risk', 'data', 'years', 'level', 'impact', 'model', 'relationship', 'age', 'people', 'population'
                ,'water', 'samples', 'compounds', 'concentration', 'concentrations', 'production', 'content', 'results', 'conditions', 'products', 'treatment', 'plant', 'process', 'activity', 'extraction', 'effect', 'plants', 'acid', 'soil', 'oil'
                ]
      ,'Score' : ['0.030', '0.010', '0.009', '0.008', '0.008', '0.007', '0.007', '0.007', '0.007', '0.007', '0.007', '0.007', '0.006', '0.006', '0.005', '0.005', '0.005', '0.005', '0.005', '0.005'
                 ,'0.018', '0.012', '0.009', '0.009', '0.007', '0.007', '0.007', '0.006', '0.006', '0.006', '0.006', '0.006', '0.006', '0.006', '0.005', '0.005', '0.005', '0.005', '0.005', '0.005'
                 ,'0.016', '0.011', '0.011', '0.010', '0.010', '0.009', '0.009', '0.008', '0.008', '0.008', '0.008', '0.007', '0.007', '0.007', '0.007', '0.006', '0.006', '0.006', '0.006', '0.006'
                 ,'0.012', '0.010', '0.010', '0.008', '0.007', '0.007', '0.007', '0.007', '0.006', '0.006', '0.005', '0.005', '0.005', '0.005', '0.005', '0.005', '0.005', '0.004', '0.004', '0.004'
                 ,'0.077', '0.024', '0.015', '0.012', '0.012', '0.011', '0.011', '0.009', '0.009', '0.009', '0.008', '0.008', '0.008', '0.008', '0.008', '0.007', '0.006', '0.006', '0.006', '0.005'
                 ,'0.015', '0.011', '0.009', '0.009', '0.009', '0.008', '0.007', '0.007', '0.007', '0.006', '0.006', '0.006', '0.005', '0.005', '0.005', '0.005', '0.005', '0.005', '0.005', '0.004'
                 ,'0.019', '0.014', '0.014', '0.013', '0.011', '0.010', '0.009', '0.008', '0.008', '0.008', '0.008', '0.007', '0.007', '0.007', '0.006', '0.006', '0.006', '0.005', '0.005', '0.005'
                 ,'0.029', '0.028', '0.022', '0.017', '0.011', '0.010', '0.010', '0.009', '0.009', '0.008', '0.007', '0.007', '0.007', '0.006', '0.006', '0.006', '0.006', '0.006', '0.006', '0.005'
                 ,'0.017', '0.014', '0.009', '0.009', '0.008', '0.008', '0.007', '0.007', '0.007', '0.007', '0.007', '0.007', '0.007', '0.006', '0.006', '0.006', '0.006', '0.006', '0.005', '0.005'
                 ,'0.016', '0.013', '0.011', '0.011', '0.010', '0.009', '0.009', '0.009', '0.009', '0.008', '0.008', '0.007', '0.007', '0.007', '0.007', '0.006', '0.006', '0.006', '0.006', '0.006'
                 ]})

In [37]:
df_word_cloud_viz.head(100)

,Assigned Topic,Word,Score
0,0,species,0.030
1,0,water,0.010
2,0,areas,0.009
3,0,climate,0.008
4,0,results,0.008
...,...,...,...
95,4,mortality,0.007
96,4,patient,0.006
97,4,survival,0.006
98,4,surgery,0.006


Lets save it to a csv file.

In [38]:
df_word_cloud_viz.to_csv("./data_frame_word_cloud_viz.csv", sep=',', index=False )